In [1]:
import pandas as pd
import numpy as np
import sqlite3
import os.path
from sqlalchemy import create_engine
from matplotlib import pyplot as plt
import pickle as pk
import qgrid
import collections

pd.set_option("display.max_rows",15)
%matplotlib inline

In [2]:
shape_of_network = [2,1,1]
num_of_layers_dense = 2

fi = open("x_y_input_traffic_shop_id_1.pkl","rb")
data = pk.load(fi)
x_train, x_valid, x_test, y_train, y_valid, y_test =  map(data.get, ('x_train', 'x_valid', 'x_test', 'y_train', 'y_valid', 'y_test'))
fi.close()

In [3]:
import tensorflow as tf

In [4]:
graph_dense = tf.Graph()
with graph_dense.as_default():
    with tf.name_scope("dense_inputs"):
        x = tf.placeholder(dtype = tf.float64, shape = [None, shape_of_network[0]], name="x")
        y_ = tf.placeholder(dtype = tf.float64, shape = [None, shape_of_network[-1]], name = "y_")

    y_h = x
    with tf.name_scope("dense_hidden"):
        w_h = tf.Variable(tf.truncated_normal(shape = [shape_of_network[0], shape_of_network[1]], dtype = tf.float64), name = "hidden_weights")
        tf.summary.histogram(name="hidden_weights_0", values = w_h)
            
        b_h = tf.Variable(tf.constant(value = 0.1, dtype = tf.float64, shape=[shape_of_network[1]]), name="hidden_bias")
        y_h = tf.nn.relu(tf.matmul(y_h , w_h) + b_h, name="relu")
            
        for i in range(1, num_of_layers_dense):
            w_h = tf.Variable(tf.truncated_normal(shape = [shape_of_network[1], shape_of_network[1]], dtype = tf.float64), name = "hidden_weights")
            tf.summary.histogram(name="hidden_weights_{}".format(i), values = w_h)
            
            b_h = tf.Variable(tf.constant(value = 0.1, dtype = tf.float64, shape=[shape_of_network[1]]), name="hidden_bias")
        
            y_h = tf.nn.relu(tf.matmul(y_h , w_h) + b_h, name="relu")

    with tf.name_scope("dense_outputs"):
        w_o = tf.Variable(tf.truncated_normal(shape = [shape_of_network[1], shape_of_network[-1]], dtype = tf.float64), name = "output_weights")
        tf.summary.histogram(name="output_weights", values = w_o)
        b_o = tf.Variable(tf.constant(value = 0.1, dtype = tf.float64, shape=[shape_of_network[-1]]), name="output_bias")
    
        y = tf.matmul(y_h, w_o) + b_o

    with tf.name_scope("cost"):
        cost = tf.reduce_mean(
            tf.abs(
                tf.divide(
                    tf.subtract(y, y_),
                    tf.add(y, y_)
                )
            )
        , name = "cost")
        tf.summary.scalar(name="cost",  tensor = cost)

    with tf.name_scope("optimizer"):
        learning_rate=0.001
        grad_clip=5
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
        train_op = tf.train.AdamOptimizer(learning_rate)
        optimizer = train_op.apply_gradients(zip(grads, tvars))

num_of_layers_lstm = 2
rnn_size = 2
batch_size = 128

graph_lstm = tf.Graph()
with graph_lstm.as_default():
    with tf.name_scope("lstm_inputs"):
        lstm_x = tf.placeholder(dtype = tf.float64, shape = [None, shape_of_network[0]], name="x")
        lstm_y_ = tf.placeholder(dtype = tf.float64, shape = [None, shape_of_network[1]], name = "y_")
        lstm_keep_prob = tf.placeholder(dtype = tf.float32)
        
    with tf.name_scope("lstm_cells"):
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)

        cell = tf.contrib.rnn.MultiRNNCell([drop] * num_of_layers_lstm)
        
        initial_state = cell.zero_state(batch_size, tf.float32)
        initial_state = tf.identity(initial_state,name="initial_state")
    
    with tf.name_scope("lstm_rnn"):
        outputs, state = tf.nn.dynamic_rnn(cell, x, dtype = tf.float32)
        final_state = state
        final_state = tf.identity(final_state, "final_state")


In [5]:
with tf.Session(graph = graph_dense) as session:
    session.run(tf.global_variables_initializer())
    merged = tf.summary.merge_all()
    file_writer_train = tf.summary.FileWriter("./logs/ijcai_2017/low_level_api/train", session.graph)
    file_writer_validation = tf.summary.FileWriter("./logs/ijcai_2017/low_level_api/validation")

    for i in range(5000):
        _, train_loss, train_summary = session.run([optimizer, cost, merged], feed_dict = {x:x_train, y_:y_train})
        valid_loss, valid_summary = session.run([cost, merged],feed_dict = {x:x_valid, y_:y_valid})

        if(i % 500 == 0):
            print("epoch: {:>4} training {:5.4f} validation {:5.4f}".format(i, train_loss, valid_loss))
            file_writer_train.add_summary(train_summary, i)
            file_writer_validation.add_summary(valid_summary, i)
        

epoch:    0 training 16.1264 validation 0.8034
epoch:  500 training 3.0750 validation 0.7884
epoch: 1000 training 3.0750 validation 0.7884
epoch: 1500 training 3.0750 validation 0.7884
epoch: 2000 training 3.0750 validation 0.7884
epoch: 2500 training 3.0750 validation 0.7884
epoch: 3000 training 3.0750 validation 0.7884
epoch: 3500 training 3.0750 validation 0.7884
epoch: 4000 training 3.0750 validation 0.7884
epoch: 4500 training 3.0750 validation 0.7884
